In [1]:
import os
import numpy as np
import torch
from nilearn import plotting
import data
import visualize
from words import make_classifications, Predictions
from data import normalize_fmri_data, unnormalize_fmri_data, analyze_results
from LEM import extract_data_features, predAccuracy
from numpy.linalg import norm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree

/home/vislab-001/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class argObj:
    def __init__(self, data_dir, parent_submission_dir, subj):
        self.subj = format(subj, '02')
        self.data_dir = os.path.join(data_dir, 'subj' + self.subj)
        self.parent_submission_dir = parent_submission_dir
        self.subject_submission_dir = os.path.join(self.parent_submission_dir,
                                                   'subj' + self.subj)



In [3]:
words2 = ['furniture', 'food', 'kitchenware', 'appliance', 'person', 'animal', 'vehicle', 'objects',
            'electronics', 'sports', 'traffic', 'outdoor', 'home', 'plumbing', 'computer']

In [4]:
platform = 'jupyter_notebook'
device = 'cuda'
device = torch.device('cuda')
# setting up the directories and ARGS
data_dir = ''#../MQP/algonauts_2023_challenge_data/'
parent_submission_dir = '../submission'
subj = 1 # @param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}
# args

args = argObj(data_dir, parent_submission_dir, subj)
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

words = ['furniture', 'food', 'kitchenware', 'appliance', 'person', 'animal', 'vehicle', 'accessory',
            'electronics', 'sports', 'traffic', 'outdoor', 'home', 'clothing', 'hygiene', 'toy', 'plumbing', 
             'computer', 'fruit', 'vegetable', 'tool']


In [5]:
print("________ Process Data ________")

# Normalize Data Before Split
lh_fmri, lh_data_min, lh_data_max = normalize_fmri_data(lh_fmri)
rh_fmri, rh_data_min, rh_data_max = normalize_fmri_data(rh_fmri)

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

train_img_dir = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir = os.path.join(args.data_dir, 'test_split', 'test_images')
train_img_list = os.listdir(train_img_dir)
train_img_list = train_img_list[: 4500]
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()

print('\nTraining images: ' + str(len(train_img_list)))
print('\nTest images: ' + str(len(test_img_list)))


________ Process Data ________
LH training fMRI data shape:
(9841, 19004)
(Training stimulus images × LH vertices)

RH training fMRI data shape:
(9841, 20544)
(Training stimulus images × RH vertices)

Training images: 9841

Test images: 159


In [6]:
print("________ Split Data ________")

idxs_train, idxs_val, idxs_test = data.splitdata(train_img_list, test_img_list, train_img_dir)
lh_fmri_train = lh_fmri[idxs_train]
rh_fmri_train = rh_fmri[idxs_train]
lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_val = rh_fmri[idxs_val]

________ Split Data ________
Training stimulus images: 8857

Validation stimulus images: 984

Test stimulus images: 159


In [7]:
print("________ Make Lists ________")

train_images = data.makeList(train_img_dir, train_img_list, idxs_train)
val_images = data.makeList(train_img_dir, train_img_list, idxs_val)
test_images = data.makeList(test_img_dir, test_img_list, idxs_test)
torch.cuda.empty_cache()

________ Make Lists ________


In [8]:
print("________ Make Classifications ________")
lh_classifications = make_classifications(train_images, idxs_train, device)
rh_classifications = lh_classifications
lh_classifications_val = make_classifications(val_images, idxs_val, device)
rh_classifications_val = lh_classifications_val
lh_classifications_test = make_classifications(test_images, idxs_test, device)
rh_classifications_test = lh_classifications_test
torch.cuda.empty_cache()

________ Make Classifications ________



0: 640x640 1 bird, 1: 640x640 4 clocks, 2: 640x640 2 persons, 2 horses, 3: 640x640 1 truck, 4: 640x640 1 cup, 17 donuts, 5: 640x640 1 broccoli, 6: 640x640 21 cars, 1 train, 2 trucks, 7: 640x640 2 persons, 1 tv, 1 laptop, 2 mouses, 1 cell phone, 8: 640x640 1 person, 1 tie, 9: 640x640 1 person, 10: 640x640 2 bottles, 1 cup, 1 chair, 11: 640x640 1 person, 2 couchs, 1 dining table, 12: 640x640 1 person, 2 surfboards, 13: 640x640 3 persons, 1 fire hydrant, 1 clock, 14: 640x640 1 person, 1 sports ball, 15: 640x640 1 person, 16: 640x640 2 persons, 5 cars, 1 bus, 17: 640x640 8 persons, 3 motorcycles, 18: 640x640 14 persons, 1 skis, 1 snowboard, 19: 640x640 14 kites, 20: 640x640 2 persons, 1 surfboard, 21: 640x640 1 person, 1 surfboard, 22: 640x640 1 person, 2 sinks, 23: 640x640 1 microwave, 1 oven, 1 refrigerator, 24: 640x640 1 car, 1 bus, 25: 640x640 3 cars, 1 stop sign, 26: 640x640 2 persons, 2 sports balls, 27: 640x640 8 persons, 3 motorcycles, 28: 640x640 10 persons, 3 umbrellas, 1 frisbe

In [9]:
print("________ Extract Image Features ________")

train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader = (
    data.transformData(train_img_dir, test_img_dir, idxs_train, idxs_val, idxs_test, 128))

features_train, features_val, features_test = (
    extract_data_features(train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader, 128))

________ Extract Image Features ________


Extracting Features: 100%|██████████| 3/3 [00:01<00:00,  1.77it/s]


Training images features:
(8857, 64)
(Training stimulus images × PCA features)

Validation images features:
(984, 64)
(Validation stimulus images × PCA features)

Test images features:
(159, 64)
(Test stimulus images × PCA features)


In [10]:
print("________ Organize Input________")

LH_train_class, LH_train_FMRI = data.organize_input(lh_classifications, features_train, lh_fmri_train)
LH_val_class, LH_val_FMRI = data.organize_input(lh_classifications_val, features_val, lh_fmri_val)    
RH_train_class, RH_train_FMRI = data.organize_input(rh_classifications, features_train, rh_fmri_train)
RH_val_class, RH_val_FMRI = data.organize_input(rh_classifications_val, features_val, rh_fmri_val)


________ Organize Input________


Linear Regression

In [11]:
print("________ Predictions Validation ________")
model = LinearRegression()
lh_fmri_val_pred = Predictions(LH_train_class, LH_train_FMRI, LH_val_class, model)
print('checkpoint')
rh_fmri_val_pred = Predictions(RH_train_class, RH_train_FMRI, RH_val_class, model)
# lh_fmri_test_pred = Predictions(LH_train_class, LH_train_FMRI, LH_test_class )
# rh_fmri_test_pred = Predictions(RH_train_class, RH_train_FMRI, RH_test_class )


________ Predictions Validation ________


In [12]:
print("________ Analyze Results ________")

analyze_results(LH_val_FMRI, lh_fmri_val_pred)
analyze_results(RH_val_FMRI, rh_fmri_val_pred)

________ Analyze Results ________
Mean Squared Error: 0.016321399213161423
Mean Absolute Error: 0.10094006137528685
R Squared Score: 0.17038510318994074
Mean Squared Error: 0.01656307924349148
Mean Absolute Error: 0.10178095906831773
R Squared Score: 0.17388916840297047


Bayesian Ridge

In [ ]:
print("________ Predictions Validation ________")
model = BayesianRidge()
lh_fmri_val_pred = Predictions(LH_train_class, LH_train_FMRI, LH_val_class, model)
print('checkpoint')
rh_fmri_val_pred = Predictions(RH_train_class, RH_train_FMRI, RH_val_class, model)
# lh_fmri_test_pred = Predictions(LH_train_class, LH_train_FMRI, LH_test_class )
# rh_fmri_test_pred = Predictions(RH_train_class, RH_train_FMRI, RH_test_class )

In [ ]:
print("________ Analyze Results ________")

analyze_results(LH_val_FMRI, lh_fmri_val_pred)
analyze_results(RH_val_FMRI, rh_fmri_val_pred)

Decision Tree Regressor

In [13]:
print("________ Predictions Validation ________")
model = tree.DecisionTreeRegressor()
lh_fmri_val_pred = Predictions(LH_train_class, LH_train_FMRI, LH_val_class, model)
print('checkpoint')
rh_fmri_val_pred = Predictions(RH_train_class, RH_train_FMRI, RH_val_class, model)
# lh_fmri_test_pred = Predictions(LH_train_class, LH_train_FMRI, LH_test_class )
# rh_fmri_test_pred = Predictions(RH_train_class, RH_train_FMRI, RH_test_class )

________ Predictions Validation ________


In [ ]:
print("________ Analyze Results ________")

analyze_results(LH_val_FMRI, lh_fmri_val_pred)
analyze_results(RH_val_FMRI, rh_fmri_val_pred)

In [ ]:
print("________ Unnormalize Data ________")

lh_fmri_val_pred = unnormalize_fmri_data(lh_fmri_val_pred, lh_data_min, lh_data_max)
rh_fmri_val_pred = unnormalize_fmri_data(rh_fmri_val_pred, rh_data_min, rh_data_max)

In [ ]:
print("________ Re-Load Data ________")
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_val = rh_fmri[idxs_val]


In [ ]:
print("________ Prediction Accuracy ________")

lh_correlation, rh_correlation = predAccuracy(lh_fmri_val_pred, lh_fmri_val, rh_fmri_val_pred, rh_fmri_val)


In [ ]:
print("________ Visualize Each Class ________")

length = len(words)
for clss in range(length):
    avg_lh_pred = []
    avg_lh_real = []
    avg_rh_pred = []
    avg_rh_real = []
    for i in range(len(lh_classifications_val)):

        if lh_classifications_val[i][1] == clss:
            avg_lh_pred.append(lh_fmri_val_pred[i])
            avg_lh_real.append(lh_fmri_val[i])

        if rh_classifications_val[i][1] == clss:
            avg_rh_pred.append(rh_fmri_val_pred[i])
            avg_rh_real.append(rh_fmri_val[i])
    # Only look at classes that are observed 
    if(len(avg_lh_pred) < 10):
        pass
    else:
        print(words[clss])
        lh = np.mean(avg_lh_pred, axis=0)
        rh = np.mean(avg_rh_pred, axis=0)
        print("MEAN PRED LH:\n", lh)
        print("MEAN PRED RH:\n", rh)
        visualize.plot_predictions(args, lh, rh, 'left')
        visualize.plot_predictions(args, lh, rh, 'right')
        plotting.show()
        lh2 = np.mean(avg_lh_real, axis=0)
        rh2 = np.mean(avg_rh_real, axis=0)
        print("MEAN REAL LH:\n", lh2)
        print("MEAN REAL RH:\n", rh2)
        visualize.plot_predictions(args, lh2, rh2, 'left')
        visualize.plot_predictions(args, lh2, rh2, 'right')
        plotting.show()
        corr = np.corrcoef(avg_lh_pred, avg_lh_real)
        print(len(avg_lh_pred))
        print("Corre ", np.mean(corr))

print("________ END ________")